Shape Analysis
====
This notebook runs through the shape analysis; the segmentation needs to already have been performed.

We can do this using two dimensionality-reduction techniques: **PCA** or **LDA**.

1 - PCA
----
PCA is a standard dimensionality reduction technique that is often used on high-dimensional data.

<details>
<summary>High dimensional data?</summary>
<div class="alert alert-block alert-info">
Our Elliptic Fourier Analysis represents each scale as a list of numbers; the length of this list is the dimensionality of our space.  

Here, "dimensionality" just means "degrees of of freedom" - how many independent parameters we have describing each scale.  

If we use 200 numbers to describe the shape of each scale, then our Elliptic Fourier Space is 200-dimensional.

Applying PCA or LDA to these high-dimensional lists of numbers will find ways to represent them as lower-dimensional (read: shorter) lists of numbers.
</div>
</details>

PCA returns the "principal components" (PCs) in our space - the axes along which the dataset varies most.
PCA is a powerful technique because it is:
1. **A linear transformation**: the PCs are an interpretable combination of the original features
2. **Based on global variance**: they capture the dominant patterns of variation across the whole dataset.

We need point 1 for our shape analysis - we want our lists of numbers to be interpretable in terms of biological features (e.g. size, aspect ratio, ...).

Point 2 means that PCA gives the best low-dimensional summary of the *whole* dataset - this is not necessarily the same thing as the low-dimensional summary that best separates between classes.

In [ ]:
"""
First, read in the data
"""

import pathlib

parent_dir = pathlib.Path(
    "~/zebrafish_rdsf/Carran/Postgrad/segmentations_cleaned"
).expanduser()
assert parent_dir.exists()

segmentation_paths = sorted([str(x) for x in parent_dir.glob("*.tif")])
f"{len(segmentation_paths)} segmentations"

In [ ]:
"""
Read in metadata, including image filepaths
"""

import numpy as np

from scale_morphology.scales import metadata

df = metadata.df([str(x) for x in segmentation_paths])
df = df.drop(columns="no_scale")

assert len(df) == 928, "Did the number of scales change?"
print(len(df), "scales after dropping empty ones")
df.head()

In [ ]:
"""
If necessary, read in the scales from the RDSF and perform EFA on them.

Otherwise read in the EFA coefficients from a cache
"""

import numpy as np

from scale_morphology.scales import efa


coeff_dump = pathlib.Path("carran_coeffs.npy")

if coeff_dump.is_file():
    coeffs = np.load(coeff_dump)
else:
    n_edge_points, order = 300, 50
    coeffs = efa.run_analysis(
        df["path"],
        df["magnification"],
        n_points=n_edge_points,
        order=order,
        n_threads=32,
    )

    np.save(coeff_dump, coeffs)

Now that we've found the EFD descriptors for our scales, we can perform PCA to find which features are most important for describing the variation in our dataset.

In [ ]:
"""
Perform PCA

We won't scale the features before PCA, because we want them to keep their original
magnitudes - if we scale the features to have a standard mean/std, then we will
artificially inflate the importance of the higher harmonics.
"""

from sklearn.decomposition import PCA

# We can choose any number of PCs to extract from our features
# We will want at least enough to describe the variation in the dataset
n_components = 10

pca = PCA(n_components=n_components)
pca_coeffs = pca.fit_transform(coeffs)

In [ ]:
"""Visualise the PCA coeffs"""

from matplotlib import colors
import matplotlib.pyplot as plt


def pca_barplot(scalings):
    """
    Make a bar plot showing the contribution of the features to each PC
    """
    # Want to plot the first two features (size, d_1) and then a whole number
    # of ellipses, each of which are made up of 4 components
    n_ellipses = 3
    n_bars = 2 + n_ellipses * 4

    n_pcs = scalings.shape[0]

    # 4 columns, with as many rows as we need
    n_col = 4
    n_row = int(np.ceil(n_pcs / n_col))
    fig, axes = plt.subplots(n_row, n_col, figsize=(n_col * 4, n_row * 4))

    # First two ticks for size, d_1; then calculate the rest + build their labels
    xticks = [-3, -1]
    xticklabels = ["size", r"$d_1$"]
    vlines = [-2, 0]
    for i in range(n_ellipses):
        xticklabels += [rf"$a_{i+2}$", rf"$b_{i+2}$", rf"$c_{i+2}$", rf"$d_{i+2}$"]

        start, end = 1 + 5 * i, 5 + 5 * i
        xticks += list(range(start, end))

        vlines.append(start - 1)

    for i, (axis, scaling) in enumerate(zip(axes.flat, scalings)):
        axis.bar(xticks, scaling[:n_bars])

        axis.set_xticks(
            xticks,
            xticklabels,
            ha="right",
        )

        # Every 4 features is one ellipse - separate them visually
        for v in vlines:
            axis.axvline(v, color="k", linestyle="--")

        axis.set_ylim(-1.1, 1.1)
        axis.set_yticks([-1, 0, 1])

        axis.set_title(f"PC{i+1}")


pca_barplot(pca.components_)

We can see here that the first two PCs correspond almost purely to the size and aspect ratio (coefficient $d_1$) respectively.

This might be expected - the size of each scale varies much more than the other features, so it makes sense that this coefficient explains a lot of the global variation.

We can also plot a heatmap of the features' contribution to the PCs; this is a slightly more compact way of visualsing the above bar charts. We will notice that the higher features (corresponding to smaller spatial scales) contribute less to the early PCs.

We can also plot the feature importance of each PC.

In [ ]:
"""
Plot heatmap of components in terms of features, and importance of components

"""
from sklearn.base import BaseEstimator


def heatmap(scalings):
    """
    Plot a heatmap showing component loadings in terms of features
    """
    fig, axis = plt.subplots(figsize=(10, 5))

    im = axis.matshow(
        scalings.T,
        aspect="auto",
        cmap="seismic",
        norm=colors.CenteredNorm(vcenter=0.0),
    )

    n_components = scalings.shape[0]
    axis.set_xticks(range(n_components), range(1, n_components + 1))
    axis.set_title("Feature contribution to each component")
    axis.set_ylabel("Component")
    axis.set_xlabel("Feature")
    fig.colorbar(im, ax=axis)


def feature_importance(estimator: BaseEstimator):
    """
    Plot feature importance of PCA or LDA
    """
    fig, axis = plt.subplots(figsize=(8, 4))

    percent_importances = estimator.explained_variance_ratio_
    axis.bar(np.arange(len(percent_importances)), percent_importances)
    axis.set_yscale("log")


heatmap(pca.components_)
feature_importance(pca)

We can see that the feature importance is dominated by the size feature (#0). I will change this in a bit to do the PCA without size...

However, when we make a scatter colour-coding our different classes, we will find that PCA does not distinguish between them very well. This is to be expected; PCA finds the components that most describe **global** variation, not variation between classes.

In [ ]:
"""
Plot 2D KDE/scatter plots showing the different classes, along the different PC axes
"""

from scale_morphology.scales import plotting

keep = df["sex"] != "?"
grouping_df = df.loc[keep, ["sex", "age"]]
grouping_colours = ["royalblue", "cornflowerblue", "brown", "red", "blue", "indianred", "lightcoral"]

plotting.pair_plot(
    pca_coeffs[keep],
    grouping_df,
    grouping_colours,
    axis_label="PC",
    normalise=True,
)

Linear Discriminant analysis (LDA) will be much better than PCA at separating our classes.

Linear Discriminant Analysis
----

### PCA vs LDA
PCA is a dimensionality-reduction method that uses the directions of greatest 
variance within our dataset as the principal components.
PCA is an unsupervised method - we don't need to tell PCA about the class labels
(which makes sense - it described global variation).
It is a powerful method, but it isn't guaranteed that the directions of greatest global
variance will also separate between our classes.
It is likely, if the separation between the classes is based on the same meaningful features
that describe the whole dataset, but it isn't guaranteed.

Linear Discriminant Analysis (LDA) however is a classifier - it can be used for dimensionality
reduction, but instead of finding the directions of greatest overall variance it finds the directions
which separate best between our classes.
It is a supervised method - we need to tell it what our class labels are so it can separate between them.
If we have $N$ classes, we can find at most $N-1$ directions to separate them.
We can use LDA as a drop-in replacement for the PCA above, and make the same plots: a bar chart
showing our LDA axes in terms of the EFD features, and scatter plots/histograms showing how the classes
are separated.

<details>
<summary>Maths</summary>
<div class="alert alert-block alert-info">
LDA is a classifier that creates linear decision boundaries (as you might expect from the name)
by fitting the conditional probabilities of each class to the densities and using Bayes' rule.

It finds the a covariance matrix and the sample means of each class,
then solves an eigenvalue problem to find the linear discriminant directions.

It is an optimal classifier if the classes are distributed as Gaussians with different means
but a shared covariance matrix.
It is a sub-optimal (but still fine) classifier if the classes are not distributed as Gaussians.
If they do not share a covariance matrix, the true best discriminant boundaries will be non-linear;
this is a problem for LDA since it can only find linear decision boundaries (Quadratic Discriminant
Analysis can deal with the case when the classes have different covariance matrices, but does not
give us nice interpretable axes of greatest variation).

The assumption that the covariance matrices for each are equal is therefore quite strong - we should
probably formally check this in each case...
</div>
</details>

In [ ]:
"""
Perform LDA on the above categories, pair plot + show that it separates better
"""

import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


def lda(
    input_coeffs: np.ndarray, label_df: pd.DataFrame
) -> tuple[LinearDiscriminantAnalysis, np.ndarray]:
    """
    Perform Linear Discriminant Analysis on a set of high_dimensional
    `input_coeffs`, given a dataframe holding the columns we want to
    label by `label_df`.

    Returns the fitted estimator + the transformed coeffs
    """
    assert len(label_df) == input_coeffs.shape[0]

    labels, uniques = pd.factorize(
        label_df.apply(lambda row: tuple(row.values), axis=1)
    )

    # Now we cannot choose how many components to use in our dimensionality reduction;
    # LDA just finds the best (N-1) axes to distinguish our N classes.
    # Technically we could use any number less than N-1, but we want to keep all of them
    lda_ = LinearDiscriminantAnalysis()
    return lda_, lda_.fit_transform(input_coeffs, labels)


_, lda_coeffs = lda(coeffs[keep], grouping_df)

plotting.pair_plot(
    lda_coeffs, grouping_df, grouping_colours, axis_label="LD", normalise=True
)

We can see that, as before, we have reduced the dimension of the space and can see some separation between the categories.
The separation is better than it was before - we can also run the LDA and PCA using different groupings, to see this effect even more clearly:

In [ ]:
# Choose labels + plot colours
keep = (df["sex"] == "M") & (df["age"] == 18)
grouping_df = df.loc[keep, ["growth", "mutation"]]
grouping_colours = ["indianred", "lightblue", "red", "blue"]

# PCA plot - don't need to re-fit PCA, since the PCA
# is independent of grouping
plotting.pair_plot(
    pca_coeffs[keep],
    grouping_df,
    grouping_colours,
    axis_label="PC",
    normalise=True,
)

# Do new LDA fit and plot the axes
lda_, lda_coeffs = lda(coeffs[keep], grouping_df)
plotting.pair_plot(
    lda_coeffs, grouping_df, grouping_colours, axis_label="LD", normalise=True
)


We can see that, even though the LDA only defines a few axes compared to the PCA, it is much better at separating out our classes.

However, we have not been very rigorous here.
It is possible that the LDA algorithm is fitting to noise - we might suspect this if our LD vectors contain large contributions from
high EFA harmonics (recall that the PCA vectors were mostly comprised of the lower harmonics):

In [ ]:
# LDA scalings are transposed wrt PCA
heatmap(lda_.scalings_.T)

We can see that, not only are the LDA components largely composed of the higher harmonics, the coefficients are huge (see the colourbar)
which suggests that we might be overfitting to noise in our high harmonics.

Let's see whether the exact same pipeline works if our "classes" are freshly generated random integers - if we are indeed overfitting,
then the LDA will be even able so separate our classes here:

In [ ]:
"""
Make a new dataframe holding just random numbers, separate these classes with LDA and plot the result
"""

# Make a df with a single column holding random ints
rng = np.random.default_rng()
grouping_df = pd.DataFrame({"random": rng.integers(0, 4, size=np.sum(keep))})

lda_, lda_coeffs = lda(coeffs[keep], grouping_df)
plotting.pair_plot(
    lda_coeffs, grouping_df, grouping_colours, axis_label="LD", normalise=True
)

We can see that, even though there should be nothing to distinguish our classes, we can perfectly separate them using LDA.

This is because the LDA axes are very noisy, as before:

In [ ]:
heatmap(lda_.scalings_.T)

A more formal and rigorous way to show this might be to run k-fold validation on our LDA classifier,
which basically just runs the LDA repeatedly on subsets of the data and checks that the results are consistent.

In [ ]:
"""
Run k-fold validation on our LDA
"""

from sklearn.feature_selection import f_classif
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict


def k_fold_lda(input_coeffs: np.ndarray, label_df: pd.DataFrame) -> None:
    """
    Run LDA on 5 folds of the given data and print the accuracy report
    """
    assert len(label_df) == input_coeffs.shape[0]

    labels, uniques = pd.factorize(
        label_df.apply(lambda row: tuple(row.values), axis=1)
    )

    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    lda_ = LinearDiscriminantAnalysis()

    scores = cross_val_score(
        lda_, input_coeffs, labels, cv=cv, scoring="balanced_accuracy"
    )
    print(f"Balanced accuracy: {scores.mean():.3f} ± {scores.std():.3f}")

    preds = cross_val_predict(lda_, input_coeffs, labels, cv=cv)
    print(classification_report(labels, preds, target_names=[str(u) for u in uniques]))

    fold_eta_sq = []
    for train_idx, test_idx in cv.split(input_coeffs, labels):
        lda_fold = LinearDiscriminantAnalysis().fit(input_coeffs[train_idx], labels[train_idx])
        ld_test = lda_fold.transform(input_coeffs[test_idx])
        f_vals, _ = f_classif(ld_test, labels[test_idx])

        df_between = len(np.unique(labels[test_idx])) - 1
        df_within = len(test_idx) - len(np.unique(labels[test_idx]))
        eta_sq = (df_between * f_vals) / (df_between * f_vals + df_within)
        fold_eta_sq.append(eta_sq)

    fold_eta_sq = np.stack(fold_eta_sq)
    print(f"LD etasq score mean ± std:")
    for x in fold_eta_sq.T:
        print("\t", x.mean(), "+-", x.std())



k_fold_lda(coeffs[keep], grouping_df)

With four equally sized categories, we'd expect chance performance of around 25% accuracy (which is approximately what we get).

This is bad - it's telling us that our LDA isn't fitting to signal, it's fitting to noise.

Using PCA then LDA
----
Earlier, I mentioned that PCA is likely to pick out meaningful features since it finds the axes of greatest global variation.

We can exploit this to make the LDA more stable - first we do PCA, then we run LDA on the PCA transformed coefficients.
This will avoid the overfitting issue, since by truncating the PCs we feed to the LDA algorithm we should retain most of the
signal while discarding the noise.

In [ ]:
"""
Run LDA on the PCA-transformed coefficients
"""

keep = (df["sex"] == "M") & (df["age"] == 18)
grouping_df = df.loc[keep, ["growth", "mutation"]]
grouping_colours = ["indianred", "lightblue", "red", "blue"]

# Do new LDA fit, this time to the PCA coeffs, and plot the axes
lda_, lda_coeffs = lda(pca_coeffs[keep], grouping_df)
plotting.pair_plot(
    lda_coeffs, grouping_df, grouping_colours, axis_label="LD", normalise=True
)

k_fold_lda(pca_coeffs[keep], grouping_df)

Our separation is not as good as when we fed all the features into the LDA, but our performance is still better than random chance.

We can plot a heatmap of the LDA axes projected onto the original features... TODO fill this in

In [ ]:
"""
Project from LDA -> PCA -> features and show the heatmap
"""


def lda2pca2features(pca_scalings: np.ndarray, lda_scalings: np.ndarray) -> np.ndarray:
    """
    The PCA and LDA loadings, return a 2d array giving the LDA loadings
    in terms of the original features
    """
    return pca_scalings.T @ lda_scalings

heatmap(lda2pca2features(pca.components_, lda_.scalings_).T)

The LDA loadings are now primarily in the lower features, which is what one would expect since these are what make up the PCA components.

These are the features which separate our classes best.

To interpret what they mean in terms of biological features (i.e. shape), we can plot the outline of selected scales on the scatter plots, for both PCA:

In [ ]:
plotting.shape_plot(lda_coeffs, scale_paths=df.loc[keep, "path"].values, axis_label="LD")

And LDA:

In [ ]:
plotting.shape_plot(pca_coeffs, scale_paths=df["path"], axis_label="PC")

We can run this analysis with many different labellings; some examples:

In [ ]:
keep = (df["sex"] != "?") & (df["growth"] == np.inf) & (df["mutation"] == "WT")
grouping_df = df.loc[keep, ["sex", "age"]]
k_fold_lda(pca_coeffs[keep], grouping_df)

In [ ]:
keep = (df["sex"] != "?") & (df["mutation"] == "WT") & (df["growth"] == 10.0)
grouping_df = df.loc[keep, ["sex", "age"]]
k_fold_lda(pca_coeffs[keep], grouping_df)

In [ ]:
keep = df["sex"] != "?"
grouping_df = df.loc[keep, ["sex", "age"]]
k_fold_lda(pca_coeffs[keep], grouping_df)

In [ ]:
keep = (df["sex"] == "M") & (df["age"] == 18)
grouping_df = df.loc[keep, ["growth", "mutation"]]
k_fold_lda(pca_coeffs[keep], grouping_df)

In [ ]:
keep = (df["mutation"] == "WT") & (df["sex"] == "F")
grouping_df = df.loc[keep, ["age"]]
k_fold_lda(pca_coeffs[keep], grouping_df)

In [ ]:
keep = (df["mutation"] == "WT") & (df["sex"] == "M")
grouping_df = df.loc[keep, ["age"]]
k_fold_lda(pca_coeffs[keep], grouping_df)